In [1]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister
from qiskit_aer import AerSimulator, Aer
from qiskit.quantum_info import Pauli, SparsePauliOp, Operator, Statevector
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from scipy.linalg import expm, fractional_matrix_power
from qiskit.compiler import transpile

In [2]:
# Define parameters
U_range = np.linspace(1,10,10) # Range of U values
V_init = 10.0  # Initial guess for V


In [3]:
def get_siam_hamiltonian(U,V):
    # Define the Hamiltonian using tensor products of Pauli operators
    H = SparsePauliOp.from_list([("ZIZI", U/4), ("XXII", V/2),("YYII",V/2),("IIXX",V/2),("IIYY",V/2)])
    return H.to_matrix()

# ham = get_siam_hamiltonian(1,1)
# print(ham)

In [64]:
def trotter_step(H, dt):
    return expm(-1j * dt * H)  # Trotter approx.

def greens_function(U, V, times, num_trotter_steps=20):
    H_matrix = get_siam_hamiltonian(U,V)
    #print(H_matrix)
    dt = times[1] - times[0]  # Small time step for trotterization
    U_t = trotter_step(H_matrix, dt / num_trotter_steps)  # Approximate evolution
  

    simulator = AerSimulator()
    greens_vals = []


    qc = QuantumCircuit(4)

    qc.h(0)
    initial_state = Statevector.from_instruction(qc).data.reshape(-1, 1)
    #print("initial_state", initial_state)
    
    X1 = Pauli("IIIX").to_matrix()
    #print('X1', X1)

    for t in times:
        evolved_state = U_t @ initial_state @ U_t.conj()
#         print("u conj: ", U_t.conj())
#         print("evolved state", evolved_state)
        expectation_value = np.real(np.conj(evolved_state.T) @ X1 @ evolved_state)
        #print('expectation value', expctation_value)
        greens_vals.append(expectation_value)
    #print('greens vals', greens_vals)
    return greens_vals



In [65]:
def fit_greens_function(times, greens_values):
    greens_values = np.array(greens_values, dtype=float).flatten()
    times = np.array(times, dtype=float).flatten()
    greens_values = np.nan_to_num(greens_values, nan=0.0, posinf=0.0, neginf=0.0)

    def model(t, a, w, m):
        return a * np.cos(w * t) + (1 - a) * np.cos(m * t)

    params, _ = curve_fit(model, times, greens_values)
    return params  # [a, w, m]

In [66]:
def calculate_quasiparticle_weight(V, a, w, m):
    denom = V**4 * ((a / w**4) + ((1 - a) / m**4))
    if denom == 0:
        denom = 1e-6  # Avoid division by zero
    return 1 / denom

In [67]:
def self_consistency(V, Z, tolerance=1e-6):
    return np.abs(V**2 - Z) <= tolerance

In [68]:
def dmft_routine(U_range, V_init, max_iters=50, tolerance=1e-6):
    times = np.linspace(0, 10, 50)
    Z_values = []

    for U in U_range:
        V = V_init
        for _ in range(max_iters): 
            greens_vals = greens_function(U, V, times)
            a, w, m = fit_greens_function(times, greens_vals)
            Z = calculate_quasiparticle_weight(V, a, w, m)

            if self_consistency(V, Z, tolerance):
                break 
            V = np.sqrt(Z)
        Z_values.append(Z)

    return Z_values



Z_values = dmft_routine(U_range, V_init)

plt.plot(U_range, Z_values, marker="o")
plt.xlabel("Impurity On-Site Interaction Energy (U)")
plt.ylabel("Quasiparticle Weight (Z)")
plt.title("Quasiparticle Weight vs U")
plt.grid()
plt.show()

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 16 is different from 1)